# ElasticNet Regresyon

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet, ElasticNetCV
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
df = pd.read_csv("hitters.csv")
df = df.dropna()
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [3]:
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
dms.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [4]:
y = df["Salary"]
X = df.drop(["League","Division","NewLeague","Salary"], axis = 1).astype("float64")
X = pd.concat([X, dms[["League_N","Division_W","NewLeague_N"]]], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
model = ElasticNet().fit(X_train, y_train)

* Bağımsız Değişkenler

In [5]:
model.coef_

array([ -1.86256172,   8.70489065,   5.10426375,  -2.89875799,
        -1.28642985,   5.24343682,   6.04480276,  -0.14701495,
        -0.21566628,  -0.7897201 ,   1.80813117,   0.80914508,
        -0.61262382,   0.26816203,   0.27172387,  -0.36530729,
        19.2186222 , -31.16586592,   8.98369938])

* Bağımlı Değişken

In [6]:
model.intercept_

-6.465955602113127

In [7]:
y_pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

357.1676548181244

In [8]:
r2_score(y_test, y_pred)

0.41070222469326945

# Model Doğruma (Model Tuning)

In [9]:
model_cv = ElasticNetCV(cv = 10).fit(X_train, y_train)
model_cv.alpha_

5230.7647364798695

In [10]:
model_cv.intercept_

-38.5194055839429

In [11]:
model_cv.coef_

array([ 0.62845434,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.09788752,  0.        ,
        0.27265769,  0.19270075,  0.00758665,  0.3106529 ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        ])

In [12]:
model_cv_tuned = ElasticNet(alpha = model_cv.alpha_).fit(X_train, y_train)
y_pred = model_cv_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

394.15280563218795

* Kendi oluşturacağımız lambdalar ile model kuruyoruz.

In [13]:
alphas = 10**np.linspace(10,-2,100)*0.5
model_cv_tuned = ElasticNetCV(alphas = alphas, cv = 10).fit(X_train, y_train)
y_pred = model_cv_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

393.9753065850553